In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

def normalize_string_column_values(df, column_name):
    df[column_name] = df[column_name].str.upper()
    df[column_name] = df[column_name].str.strip()
    df[column_name] = df[column_name].str.replace(' ', '_')
    return df

In [2]:
customers = pd.read_csv('data/customers.csv')

#### Limpieza y creación de columnas útiles en `customers`

In [ ]:
customers["address"] = customers["address"].str.upper()
customers.fillna({"address":"UNDEFINED"}, inplace=True)

# Los valores de "address" se caracterizan por tener en alguna parte
# 2 letras mayúsculas seguidas de un espacio y 5 dígitos.
# Interpreto las dos letras mayúsculas como el Estado y los 5 dígitos como el código postal.
pattern = r'([A-Z]{2})\s\d{5}'
customers[["state"]] = customers["address"].str.extract(pattern)
customers.fillna({"state":"N/A"}, inplace=True)

In [ ]:
customers["registration_date"] = pd.to_datetime(customers["registration_date"], format="%Y-%m-%dT%H:%M:%S.%f", errors="coerce")
# customers["registration_year_month"] = pd.DatetimeIndex(customers["registration_date"]).strftime('%Y-%m')
customers["registration_date"] = pd.DatetimeIndex(customers["registration_date"]).date
customers["last_login"] = pd.to_datetime(customers["last_login"], format="%Y-%m-%dT%H:%M:%S.%f", errors="coerce")
# customers["last_login_year_month"] = pd.DatetimeIndex(customers["last_login"]).strftime('%Y-%m')
customers["last_login"] = pd.DatetimeIndex(customers["last_login"]).date

In [ ]:
customers.fillna({
    "city":"UNDEFINED", 
    "country":"UNDEFINED", 
    "email":"UNDEFINED",
    "customer_segment": "UNDEFINED"}, inplace=True)
customers = normalize_string_column_values(customers, "customer_segment")
customers = normalize_string_column_values(customers, "country")
customers = normalize_string_column_values(customers, "city")
customers = normalize_string_column_values(customers, "email")